# Imports

In [1]:
from __future__ import print_function, division
%autosave 120

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import requests
from bs4 import BeautifulSoup
import re
import string
import time

Autosaving every 120 seconds


## Load 'IMDB' df

In [4]:
final_imdb = pickle.load(open("final_imdb.p", "rb"))

# 'Amazon Files' function: Using AmazonID, scrape Amazon pages.

In [1]:
page = 1
num = 1

for amzid in final_imdb['amazonid']:
    t0 = time.time()
    response_delay = time.time() - t0
    time.sleep(10*response_delay)  # wait 10x longer than it took them to respond
    
    website = 'https://www.amazon.com/exec/obidos/ASIN/%s/internetmoviedat/'
    url = website % amzid
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    soup =BeautifulSoup(requests.get(url, headers = header).text, "lxml")
 
    pretty_amazon = str(soup.prettify)
    pickle.dump(pretty_amazon, open("amazon_%s.p" % num, "wb"))
    
    print('Scraped page %d out of 606' % page)
    page += 1
    num += 1

## Find & remove 'Robot Check' files from 'Amazon' files

In [13]:
amazon_frame = pd.DataFrame()
amazon_frame['amazonid'] = final_imdb['amazonid']
amazon_frame['film'] = final_imdb['title']
amazon_frame['file_index'] = final_imdb.index

In [17]:
def page_header(a, b):
    df = pd.DataFrame()
    header = []
    amazonid = []
    
    for number in range(a, b):
        soup = BeautifulSoup(pickle.load(open("amazon_%d.p" %number, "rb")), "lxml")        

        item1 = str(soup.find_all('title')[0].text)
        header.append(item1)    

    df['header'] = header

    return df


In [18]:
amazon_headers = page_header(1,607)

In [20]:
amazon_frame['header'] = amazon_headers

## 'Amazon' function: From Amazon files, get Amazon ID, Stars, Reviews, Ranks, Language, Publisher

In [13]:
def amazon_page(number):

    file_df = pd.DataFrame()

    amazonid = []
    title_author = []
    stars = []
    reviews = []
    bs_rank = []
    language = []
    publisher = []

    soup = BeautifulSoup(pickle.load(open("amazon_%d.p" % number, "rb")), "lxml")
    
    try:
        item1 = str(soup.find_all('title')[0].text).rpartition(
            ':')[0].rpartition(':')[0].rpartition(':')[0]
        title_author.append(item1)
    except: 
        title_author.append(np.nan)
        #Turns out the location of the title & author in the header is inconsistent.
    
    try:
        item2 = soup.find_all('span', {'class': 'reviewCountTextLinkedHistogram'})[0].get('title')
        stars.append(item2)
    except:
        stars.append(np.nan)
    
    try:
        item3 = soup.find_all('span', {'id': 'acrCustomerReviewText'})[0].text
        reviews.append(item3)
    except:
        reviews.append(np.nan)

    try:     
        item4 = soup.findAll(attrs={"id" : "amsDetailRightPBookTall"})[0].get('data-detailpageasin')
        amazonid.append(item4)
    except:
        amazonid.append(np.nan)
    
    try: 
        item5 = soup.find_all('li', {'id': 'SalesRank'})[0].find('span').text
        bs_rank.append(item5)    
    except:
        bs_rank.append(np.nan)

    try: 
        for child in soup.find_all('td', {'class' : 'bucket'})[0].find_all('li'):
            if "Language" in str(child.text):
                language.append(str(child.text))
    except:
        language.append(np.nan) 

    try: 
        for child in soup.find_all('td', {'class' : 'bucket'})[0].find_all('li'):
            if "Publisher" in str(child.text):
                publisher.append(str(child.text))
    except:
        publisher.append(np.nan) 
        
    del soup

    file_df['amazonid'] = amazonid
    file_df['title_author'] = title_author
    file_df['stars'] = stars
    file_df['reviews'] = reviews
    file_df['bs_rank'] = bs_rank
    file_df['language'] = language
    file_df['publisher'] = publisher
    
    
    return file_df

In [ ]:
amazon_list = []

In [ ]:
page = 1
for number in range (1, 531):
    df = amazon_page(int(number))
    header_list.append(df)
    print('Scraping file %d out of 530' % page)
    page +=1

In [37]:
final_amazon = pd.concat(amazon_list)

In [55]:
clean_amazon = final_amazon.dropna()

## 'Amazon' function: From Amazon files, rescrape header

In [62]:
# rescrape header without any rpartition.
def amazon_header(number):

    file_df = pd.DataFrame()

    amazonid = []
    header = []

    soup = BeautifulSoup(pickle.load(open("amazon_%d.p" % number, "rb")), "lxml")
    
    try:
        item1 = str(soup.find_all('title')[0].text)
        header.append(item1)
    except: 
        title_author.append(np.nan)
    
    try:     
        item4 = soup.findAll(attrs={"id" : "amsDetailRightPBookTall"})[0].get('data-detailpageasin')
        amazonid.append(item4)
    except:
        amazonid.append(np.nan)
    
    del soup

    file_df['amazonid'] = amazonid
    file_df['header'] = header
    
    
    return file_df

In [63]:
header_list = []

In [ ]:
page = 1
for number in range (1, 531):
    df = amazon_header(int(number))
    header_list.append(df)
    print('Scraping file %d out of 530' % page)
    page +=1

In [68]:
final_header = pd.concat(header_list)

In [73]:
final_header = final_header.dropna()

In [74]:
final_header.shape

(503, 2)

## Merge 'Amazon' df & 'clean header' df

In [76]:
cleanish_final = final_header.merge(clean_amazon, on='amazonid', how='left')

In [83]:
pickle.dump(cleanish_final, open("cleanish_amazon.p", "wb"))

# Clean 'Amazon' df

In [ ]:
def feature_cleaner(feature):
    empty_list = []
    for s in cleaner_amazon[feature]:
        s = re.sub("[^0-9]", "", s)
        empty_list.append(s)
    return empty_list

## stars, reviews, ranks, publishers

In [4]:
clean_amazon = pickle.load(open("cleanish_amazon.p", "rb"))

In [26]:
cleaner_amazon = clean_amazon.dropna(axis=0, how='any', subset= [
    'stars', 'reviews', 'bs_rank', 'language'], inplace=False)

In [39]:
cleaner_amazon['reviews'] = feature_cleaner('reviews')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [40]:
cleaner_amazon['bs_rank'] = feature_cleaner('bs_rank')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [51]:
stars = []
for s in cleaner_amazon['stars']:
    s = s.replace(' out of 5 stars', '')
    stars.append(s)
cleaner_amazon['stars'] = stars

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [74]:
years = []
for s in cleaner_amazon['publisher']:
    s = re.sub("[^0-9]", "", s)[-4:]
    years.append(s)
cleaner_amazon['years'] = years

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


## Clean Publishers
### Smaller publishers

In [115]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Publisher: ', '')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [116]:
cleaner_amazon['publisher'] = cleaner_amazon['publisher'].apply(lambda x: x.split('(')[0]).apply(lambda x: x.split(';')[0])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [136]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.strip()
    s = s.replace('.', '').replace(',', '').replace(' and ', ' & ')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [146]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Bantam Press', 'Bantam').replace('Bantam Spectra', 'Bantam')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [214]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace("G P Putnam's Sons", 'G P Putnam').replace(
        "GP Putnam's Sons", 'G P Putnam').replace(
        'Putnam Adult', 'G P Putnam').replace(
        'Putnam', 'G P Putnam').replace(
        'G P G P Putnam', 'G P Putnam')
     
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [163]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Bloomsbury Children\'s Books', 'Bloomsbury').replace(
        'Bloomsbury Publishing', 'Bloomsbury').replace(
        'Bloomsbury Publishing PLC', 'Bloomsbury').replace(
        'Bloomsbury USA', 'Bloomsbury').replace('Bloomsbury PLC', 'Bloomsbury')
    
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [148]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Dutton Adult', 'Dutton').replace('Dutton Books', 'Dutton')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [149]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Delacorte Books for Young Readers', 'Delacorte Press')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [155]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Scholastic Inc', 'Scholastic').replace(
        'Scholastic Paperbacks', 'Scholastic').replace('Scholastic Press', 'Scholastic')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [152]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Houghton Mifflin Company', 'Houghton Mifflin').replace(
        'Houghton Mifflin Harcourt', 'Houghton Mifflin')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [154]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Signet Classics', 'Signet')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [158]:
publisher = []
for s in cleaner_amazon['publisher']:
    s = s.replace('Viking Adult', 'Viking').replace(
        'Viking Books for Young Readers', 'Viking')
    publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Big5 Publisher: Simon and Schuster's publishing divisions and imprints include Atria, Folger Shakespeare Library, Free Press, Gallery Books, Howard Books, Pocket Books, Scribner, Simon & Schuster, Threshold Editions and Touchstone.

In [211]:
publisher = []
big_five = ['SIMON & SCHUSTER LTD', 'Atria', 'Atria Books', 'Pocket', 'Pocket Books', 
            'Pocket Star', 'Scribner', 'Touchstone', 
            'Playboy Press : trade distribution by Simon & Schuster']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        s = s.replace(s, 'Simon & Schuster')
        publisher.append(s)
    else:
        publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### Big5 Publisher: As a result, Penguin Random House has nearly 250 imprints and publishing houses. Some of the most well-known Penguin Random House publishing groups are: Random House Publishing Group, Knopf Doubleday Publishing Group; Crown Publishing Group; Penguin Group U.S.; Dorling Kindersley; Mass Market Paperbacks, Penguin Group U.S.; Random House Children's Books; Penguin Young Readers Group, U.S.

In [208]:
publisher = []
big_five = ['Penguin Books Ltd', 'Penguin Classic', 'Penguin Classics', 'Penguin Press HC The', 
           'Penguin Putnam~childrens Hc', 'Random House Books for Young Readers', 
           'Random House Value Publishing', 'Alfred A Knopf', 'Knopf', 'Random House',
            'Knopf Books for Young Readers', 'Doubleday','Crown','Penguin Bookss', 'Penguin Books']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        s = s.replace(s, 'Penguin Random House')
        publisher.append(s)
    else:
        publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


### Big5 Publisher: Hachette's publishing divisions include Grand Central Publishing; Little, Brown and Company; Little, Brown and Company Books for Young Readers; Faith Words; Center Street; Orbit; Yen Press; Hachette Audio; and Hachette Digital. Read about Forever, Hachette's Romance line, and about Forever Yours, their digital first Romance line. Time Warner acquired Little, Brown in 1968 and HBG was created when Hachette Livre acquired Time Warner Book Group in 2006.

In [195]:
publisher = []
big_five = ['Little Brown & Co', 'Little Brown', 'Little Brown & Company', 'Little Brownmpany',
            'Warner Books', 'Warner',
            'Little Brown Book Group', 'Little Brown Books for Young Readers', 
            'Grand Central Publishing',]
for s in cleaner_amazon['publisher']:
    if s in big_five:
        s = s.replace(s, 'Hachette Books')
        publisher.append(s)
    else:
        publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


### Big5 Publisher: Some of HarperCollins publishers and imprints are: HaperCollins; William Morrow; Avon Books; Broadside Books; Harper Business; HarperCollinsChildrens; HarperTeen; Ecco Books; It Books; Newmarket Press; Harper One; Harper Voyager US; Harper Perennial; HarperAcademic and Harper Audio.

In [192]:
publisher = []
big_five = ['HarperCollins', 'HarperCollins Design', 'HarperCollins Perennial', 
            'HarperCollinsCollins', 'HarperCollinsFestival', 'HarperCollinsTeen', 'HarperCollinsTorch', 
            'HarperCollinscollins', "HarperCollinscollins Children's Books", 'William Morrow & Co', 
            'William Morrow', 'Ecco', 'It Books']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        s = s.replace(s, 'HarperCollins')
        publisher.append(s)
    else:
        publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


### Big5 Publisher: The Macmillan U.S. trade book publishers include Farrar, Straus and Giroux; Henry Holt and Company; Picador; St. Martin’s Press; Tor/Forge; Macmillan Audio; and Macmillan Children’s Publishing Group. Macmillan also publishes into the college and academic book marketplace. In the many of the Macmillan U.S. publishers headquartered in New York City's historic Flatiron Building.
https://www.thebalance.com/the-big-five-trade-book-publishers-2800047
Updated April 24, 2017 

In [189]:
publisher = []
big_five = ["St Martin's Griffin", "St Martin's Paperbacks", 'St Martins Pr', "St Martin's Press", 
            'Farrar Straus & Giroux', 'Henry Holt & Co', 'Picador', 'Tor Books','Forge',
            'Forge Books']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        s = s.replace(s, "Macmillan")
        publisher.append(s)
    else:
        publisher.append(s)
cleaner_amazon['publisher'] = publisher

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### Big Publishing Feature

In [217]:
big_publishing = []
big_five = ['Penguin Random House', 'Hachette Books', 'Simon & Schuster', 
                  'HarperCollins', 'Macmillan']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        big_publishing.append(1)
    else:
        big_publishing.append(0)
cleaner_amazon['big_publishing'] = big_publishing

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Dummy Publishers

In [6]:
PRH_dummy = []
big_five = ['Penguin Random House']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        PRH_dummy.append(1)
    else:
        PRH_dummy.append(0)
cleaner_amazon['PRH_dummy'] = PRH_dummy

In [8]:
H_Books_dummy = []
big_five = ['Hachette Books']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        H_Books_dummy.append(1)
    else:
        H_Books_dummy.append(0)
cleaner_amazon['H_Books_dummy'] = H_Books_dummy

In [9]:
SS_dummy = []
big_five = ['Simon & Schuster']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        SS_dummy.append(1)
    else:
        SS_dummy.append(0)
cleaner_amazon['SS_dummy'] = SS_dummy

In [10]:
HC_dummy = []
big_five = ['HarperCollins']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        HC_dummy.append(1)
    else:
        HC_dummy.append(0)
cleaner_amazon['HC_dummy'] = HC_dummy

In [11]:
Mac_dummy = []
big_five = ['Macmillan']
for s in cleaner_amazon['publisher']:
    if s in big_five:
        Mac_dummy.append(1)
    else:
        Mac_dummy.append(0)
cleaner_amazon['Mac_dummy'] = Mac_dummy

In [15]:
pickle.dump(cleaner_amazon, open("cleaner_amazon.p", "wb"))